In [1]:
import pandas as pd
import os as os
import numpy as np
import matplotlib.pyplot as plt
from ggplot import *
import json
from io import StringIO

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline  

os.chdir('c://users/grlurton/documents/niger_election_data')

In [2]:
aa = os.listdir('data/raw/tabula-RENALOC_Niger_733/')

In [3]:
def get_data(adress) :
    url = 'data/raw/tabula-RENALOC_Niger_733/' + adress
    try :
        liste_depts = pd.read_csv( url , encoding = "ISO-8859-1" , )
        if len(liste_depts.columns) < 7:
            out = float('nan')    
        if len(liste_depts.columns) >= 8 :
            out = liste_depts.reset_index()
            out = out.drop(0)
            if len(out.columns) >= 9 :
                out = out.iloc[: , [0,1,3,4,5,6,7]]
            out.columns = ['locality' , 'population' , 'hommes', 'femmes' , 'menages' , 'menages_agricoles' , 'geoloc']
            if out.loc[1 , 'locality'].__class__.__name__ in ['int64' , 'float'] :
                out = float('nan')
            return out   
        
    except (ValueError , IndexError) :
        out = float('nan') 

In [4]:
url = 'data/raw/tabula-RENALOC_Niger_733/' + 'tabula-RENALOC_Niger_733-650.csv'
try :
    liste_depts = pd.read_csv( url , encoding = "ISO-8859-1" , )
except  (ValueError):
    liste_depts = float('nan')
#liste_depts = liste_depts.reset_index()

In [5]:
order = []

for n in range(len(aa)) :
    u = aa[n].split('-')[2].split('.')[0]
    order = order + [int(u)]
order = sorted(order)

In [6]:
renaloc = []
for i in order :
    addresse = 'tabula-RENALOC_Niger_733-' + str(i) + '.csv'
    dat = get_data(addresse)
    if (dat.__class__.__name__ == 'DataFrame') :
        print(addresse)
        if (len(renaloc) == 0) :            
            renaloc = dat
        if (len(renaloc) > 0) :
            renaloc = renaloc.append(dat , ignore_index = True)

tabula-RENALOC_Niger_733-116.csv
tabula-RENALOC_Niger_733-117.csv
tabula-RENALOC_Niger_733-118.csv
tabula-RENALOC_Niger_733-119.csv
tabula-RENALOC_Niger_733-120.csv
tabula-RENALOC_Niger_733-121.csv
tabula-RENALOC_Niger_733-122.csv
tabula-RENALOC_Niger_733-123.csv
tabula-RENALOC_Niger_733-124.csv
tabula-RENALOC_Niger_733-125.csv
tabula-RENALOC_Niger_733-126.csv
tabula-RENALOC_Niger_733-127.csv
tabula-RENALOC_Niger_733-128.csv
tabula-RENALOC_Niger_733-129.csv
tabula-RENALOC_Niger_733-130.csv
tabula-RENALOC_Niger_733-131.csv
tabula-RENALOC_Niger_733-132.csv
tabula-RENALOC_Niger_733-133.csv
tabula-RENALOC_Niger_733-134.csv
tabula-RENALOC_Niger_733-135.csv
tabula-RENALOC_Niger_733-136.csv
tabula-RENALOC_Niger_733-137.csv
tabula-RENALOC_Niger_733-138.csv
tabula-RENALOC_Niger_733-139.csv
tabula-RENALOC_Niger_733-140.csv
tabula-RENALOC_Niger_733-141.csv
tabula-RENALOC_Niger_733-142.csv
tabula-RENALOC_Niger_733-143.csv
tabula-RENALOC_Niger_733-145.csv
tabula-RENALOC_Niger_733-146.csv
tabula-REN

In [7]:
renaloc['level']  = renaloc['region'] = renaloc['departement'] = renaloc['commune'] = renaloc['milieu'] = \
        region = departement = commune = nom_sup = level = ''
for i in range(1,len(renaloc)) :
    
    u = renaloc.iloc[i]
    name = u.locality
    try :
        splitted = name.split(':')
        if (len(splitted) >= 2) :
            splitted[0] = splitted[0].replace(' ' , '')
            if (splitted[0] == 'REGIONDE') :
                renaloc.loc[i,'level']= level = 'Region'
                region = splitted[1]
                renaloc.loc[i,'region'] = region
            if (splitted[0] == 'DEPARTEMENTDE') :
                renaloc.loc[i,'level']= level = 'Departement'
                departement = splitted[1]
                
                renaloc.loc[i,'region'] = region
                renaloc.loc[i,'departement'] = departement
            if (splitted[0] == 'COMMUNEDE') :
                renaloc.loc[i,'level']= level = 'Commune'
                commune = splitted[1]
                
                renaloc.loc[i,'region'] = region
                renaloc.loc[i,'departement'] = departement
                renaloc.loc[i,'commune'] = commune
            if (splitted[1] == ' Urbain') :
                renaloc.loc[i ,'milieu'] = 'Urbain'
            if (splitted[1] == ' Rural') :
                renaloc.loc[i, 'milieu'] = 'Rural'
            if (level == 'Region'):
                renaloc.loc[i , 'region'] = region
                
                renaloc.loc[i , 'level'] = level
            if (level == 'Departement') :
                renaloc.loc[i , 'region'] = region
                renaloc.loc[i , 'departement'] = departement
                
                renaloc.loc[i , 'level'] = level
            if (level == 'Commune') :
                renaloc.loc[i , 'region'] = region
                renaloc.loc[i , 'departement'] = departement
                renaloc.loc[i , 'commune'] = commune
                
                renaloc.loc[i , 'level'] = level
        else :
            renaloc.loc[i , 'level'] = 'Localite'
            renaloc.loc[i , 'region'] = region
            renaloc.loc[i , 'departement'] = departement
            renaloc.loc[i , 'commune'] = commune
    except (RuntimeError, TypeError, NameError , AttributeError):
        pass

In [8]:
def conversion(old):
    new = old.replace(u'°',' ').replace('\'',' ').replace('"',' ')
    new = new.split()
    new_dir = new.pop()
    new.extend([0,0,0])
    return (int(new[0])+int(new[1])/60.0+int(new[2])/3600.0)

In [9]:
def extract_gps(pdf_string):
    long = pdf_string.split(';')[0]
    
    coord1 = long.split(':')[1].split("Â°")[0]
    coord2 = long.split('Â')[1].split(',')[0]
    coord3 = long.split(',')[1].split(';')[0]
    
    long = coord1 + coord2 + coord3
    
    long = long.replace('\\' , '').replace('"','').replace(' ',"")
    
    lat = pdf_string.split(';')[1]
    coord4 = lat.split("Â°")[0]
    coord4 = coord4.replace("l" , "").replace(':','')
    coord5 = lat.split('Â')[1].split(",")[0]
    coord6 = lat.split(',')[1].split(")")[0]
    
    lat =  coord4 + coord5 + coord6
    
    return [long , lat]

In [10]:
ren_save = renaloc

In [14]:
def float_all(data):
    if (data.__class__.__name__ != 'float') :
        try :
            data = data.split('\r')[0]
            data = float(data)
        except (ValueError) :
            data = float('nan')

    return data

In [15]:
renaloc = ren_save

renaloc['longitude'] = renaloc['latitude'] = ''
#renaloc = renaloc.reset_index()

num_variables = ['hommes' , 'femmes' , 'menages' , 'menages_agricoles' , 'population']

In [19]:
for i in range(len(renaloc)):
    for var in range(len(num_variables)):
        variable = num_variables[var]
        renaloc.loc[i , variable] = float_all(renaloc.loc[i , variable])
    
    
    gps = renaloc.loc[i , 'geoloc']
    if pd.isnull(gps) == False :
        try :
            gps_list = extract_gps(gps)
            if (conversion(gps_list[0]) > 0) & (conversion(gps_list[1]) > 10) :
                renaloc.loc[i , 'longitude'] = conversion(gps_list[0])
                renaloc.loc[i , 'latitude'] = conversion(gps_list[1])
        except (IndexError) :
            print('Index Error at ' + str(i))

Index Error at 2480
Index Error at 4492
Index Error at 25015
Index Error at 26924
Index Error at 29359
Index Error at 30660


In [46]:
geolocalized_data = renaloc[~(renaloc.longitude == '')]

In [30]:
renaloc.iloc[25000]

locality                                       SOKERE
population                                        357
hommes                                            175
femmes                                            182
menages                                            31
menages_agricoles                                  30
geoloc               (L:001Â°33',554 ; l:12Â°43',140)
level                                        Localite
region                                      TILLABERI
departement                                    TORODI
commune                                     MAKALONDI
milieu                                               
longitude                                        1.55
latitude                                      12.7167
Name: 25000, dtype: object

In [49]:
from bokeh.io import output_file, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool , BoxZoomTool
)

map_options = GMapOptions(lat=12.8, lng=5, map_type="hybrid", zoom=5)

plot = GMapPlot(
    x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options
)
plot.title = "Austin"

source = ColumnDataSource(
    data=dict(
        lat=geolocalized_data.latitude,
        lon=geolocalized_data.longitude,
    )
)

circle = Circle(x="lon", y="lat", size=2, fill_color="blue", fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)

plot.add_tools(PanTool(), WheelZoomTool(), BoxZoomTool())
output_file("gmap_plot2.html")
show(plot)

In [40]:
help(GMapPlot)

Help on class GMapPlot in module bokeh.models.map_plots:

class GMapPlot(MapPlot)
 |  A Bokeh Plot with a `Google Map`_ displayed underneath.
 |  
 |  Data placed on this plot should be specified in decimal lat long coordinates e.g. 37.123, -122.404.
 |  It will be automatically converted into the web mercator projection to display properly over
 |  google maps tiles.
 |  
 |  .. _Google Map: https://www.google.com/maps/
 |  
 |  Method resolution order:
 |      GMapPlot
 |      MapPlot
 |      bokeh.models.plots.Plot
 |      bokeh.models.component.Component
 |      bokeh.model.Model
 |      bokeh.core.properties.HasProps
 |      bokeh.util.callback_manager.CallbackManager
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  get_class(view_model_name) from builtins.type
 |      Given a __view_model__ name, returns the corresponding class
 |      object
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  

In [1]:
from notebook.nbconvert

ImportError: No module named 'notebook.extensions'

In [11]:
check_nbextension('usability/codefolding/main/pre_codefolding')

False

In [2]:
import sys
sys.executable

'C:\\Users\\grlurton\\AppData\\Local\\Continuum\\Anaconda3\\python.exe'